## Work
1. 請比較 SGD optimizer 不同的 momentum 及使用 nesterov 與否的表現

In [1]:
import os
import keras

# 本作業可以不需使用 GPU, 將 GPU 設定為 "無" (若想使用可自行開啟)
os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [2]:
train, test = keras.datasets.cifar10.load_data()

In [3]:
## 資料前處理
def preproc_x(x, flatten=True):
    x = x / 255.
    if flatten:
        x = x.reshape((len(x), -1))
    return x

def preproc_y(y, num_classes=10):
    if y.shape[-1] == 1:
        y = keras.utils.to_categorical(y, num_classes)
    return y    

In [4]:
x_train, y_train = train
x_test, y_test = test

# 資料前處理 - X 標準化
x_train = preproc_x(x_train)
x_test = preproc_x(x_test)

# 資料前處理 -Y 轉成 onehot
y_train = preproc_y(y_train)
y_test = preproc_y(y_test)

In [5]:
def build_mlp(input_shape, output_units = 10, num_neurons = [512, 256, 128]):
    input_layer = keras.layers.Input(input_shape)
    
    for i, n_units in enumerate(num_neurons):
        if i == 0:
            x = keras.layers.Dense(units = n_units, activation = "relu", name="hidden_layer"+str(i+1))(input_layer)
        else:
            x = keras.layers.Dense(units = n_units, activation = "relu", name="hidden_layer"+str(i+1))(x)
    
    out = keras. layers.Dense(units = output_units, activation = "softmax", name = "output")(x)
    
    model = keras.models.Model(inputs = [input_layer], outputs = [out])
    """Code Here
    建立你的神經網路
    """
    
    return model

In [9]:
"""Code Here
設定超參數
"""
LEARNING_RATE = 0.001
EPOCHS = 50
BATCH_SIZE = 256
MOMENTUM = [0.95, 0.9, 0.8]

In [13]:
results = {}
"""Code Here
撰寫你的訓練流程並將結果用 dictionary 紀錄
"""
for m in MOMENTUM:
    
    keras.backend.clear_session()
    print(f"experiment eith momentum = {m}")
    model = build_mlp(x_train.shape[-1:])
    model.summary()
    
    optimizer = keras.optimizers.SGD(lr = LEARNING_RATE, nesterov = False, momentum = m)
    model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer = optimizer)
    
    model.fit(x_train, y_train,
             epochs = EPOCHS, 
             batch_size = BATCH_SIZE, 
             validation_data = (x_test, y_test),
             shuffle = True)
    
    #collect result
    train_loss = model.history.history['loss']
    train_acc = model.history.history['accuracy']
    
    test_loss = model.history.history['val_loss']
    test_acc = model.history.history['val_accuracy']
    
    exp_name_tag = f"exp-momentum-{m}" 
    results[exp_name_tag] = {'train_loss':train_loss, 
                            'train_accuracy':train_acc, 
                            'test_loss':test_loss,
                            'test_acc':test_acc}
                            

experiment eith momentum = 0.95
Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 3072)]            0         
_________________________________________________________________
hidden_layer1 (Dense)        (None, 512)               1573376   
_________________________________________________________________
hidden_layer2 (Dense)        (None, 256)               131328    
_________________________________________________________________
hidden_layer3 (Dense)        (None, 128)               32896     
_________________________________________________________________
output (Dense)               (None, 10)                1290      
Total params: 1,738,890
Trainable params: 1,738,890
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
196/196 [==============================] - 2s 11ms/step - loss: 2.0

196/196 [==============================] - 2s 10ms/step - loss: 0.9537 - accuracy: 0.6644 - val_loss: 1.3685 - val_accuracy: 0.5297
experiment eith momentum = 0.9
Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 3072)]            0         
_________________________________________________________________
hidden_layer1 (Dense)        (None, 512)               1573376   
_________________________________________________________________
hidden_layer2 (Dense)        (None, 256)               131328    
_________________________________________________________________
hidden_layer3 (Dense)        (None, 128)               32896     
_________________________________________________________________
output (Dense)               (None, 10)                1290      
Total params: 1,738,890
Trainable params: 1,738,890
Non-trainable params: 0
_____________

Epoch 50/50
196/196 [==============================] - 2s 10ms/step - loss: 1.1490 - accuracy: 0.5961 - val_loss: 1.3361 - val_accuracy: 0.5255
experiment eith momentum = 0.8
Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 3072)]            0         
_________________________________________________________________
hidden_layer1 (Dense)        (None, 512)               1573376   
_________________________________________________________________
hidden_layer2 (Dense)        (None, 256)               131328    
_________________________________________________________________
hidden_layer3 (Dense)        (None, 128)               32896     
_________________________________________________________________
output (Dense)               (None, 10)                1290      
Total params: 1,738,890
Trainable params: 1,738,890
Non-trainable params: 0
_

196/196 [==============================] - 2s 10ms/step - loss: 1.3535 - accuracy: 0.5261 - val_loss: 1.4349 - val_accuracy: 0.4951
Epoch 50/50
196/196 [==============================] - 2s 10ms/step - loss: 1.3464 - accuracy: 0.5281 - val_loss: 1.4203 - val_accuracy: 0.5013


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
"""Code Here
將結果繪出
"""

color_bar = ["r", "g", "b", "y", "m", "k"]

plt.figure(figsize=(8,6))
for i, cond in enumerate(results.keys()):
    plt.plot(range(len(results[cond]['train_loss'])), results[cond]['train_loss'], '-', label = cond, color = color_bar[i])
    plt.plot(range(len(results[cond]['test_loss'])),results[cond]['test_loss'], '--', label = cond, color = color_bar[i])
plt.title("Loss")
plt.legend()
plt.show()

plt.figure(figsize=(8,6))
for i, cond in enumerate(results.keys()):
    plt.plot(range(len(results[cond]['train_'])), results[cond]['train_loss'], '-', label = cond, color = color_bar[i])
    plt.plot(range(len(results[cond]['test_loss'])),results[cond]['test_loss'], '--', label = cond, color = color_bar[i])
plt.title("Loss")
plt.legend()
plt.show()